In [28]:
import sys
import os
import numpy as np
import time
# Add the parent folder to sys.path
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

import pandas as pd
import density_generator
import numpy as np
import matplotlib.pyplot as plt
import qutip
import importlib
import helper_functions
import quantum_subroutines_pennylane
import pipeline_v200

from random import randint
importlib.reload(density_generator)
importlib.reload(helper_functions)
importlib.reload(quantum_subroutines_pennylane)

<module 'quantum_subroutines_pennylane' from '/home/ronin/Dev/space/QFI/quantum_subroutines_pennylane.py'>

In [29]:
def example_demo():
    """
    Demonstration: 
    We'll pick n_qubits=1 for simplicity, 
    define two small density matrices for rho, rho_delta, 
    measure sub/super fidelities, and compute 
    approximate QFI bounds for a small delta=0.1.
    """
    n_qubits = 1

    # Example: a pure state for rho
    psi = np.array([1.0, 0.0])
    rho = np.outer(psi, psi)

    # Another pure state for rho_delta
    phi = np.array([np.sqrt(0.7), np.sqrt(0.3)])
    rho_delta = np.outer(phi, phi)

    # Measure sub- and super-fidelity
    E_val, R_val = quantum_subroutines_pennylane.measure_sub_super_fidelity(n_qubits, rho, rho_delta)
    print("Subfidelity E(rho, rho_delta) =", E_val)
    print("Superfidelity R(rho, rho_delta) =", R_val)

    # Bounds on the standard fidelity: sqrt(E) <= F <= sqrt(R).
    F_lower = np.sqrt(E_val)
    F_upper = np.sqrt(R_val)
    print("Lower bound on fidelity =", F_lower)
    print("Upper bound on fidelity =", F_upper)

    # For small parameter shift delta, eq. (3) in the paper gives:
    #   I_delta(rho) = (8 / delta^2)*(1 - F(rho, rho_delta)).
    # So using the sub- and super-fidelity as bounds on F,
    # we get bounds on the QFI:
    delta = 0.1
    I_lower = (8.0 / delta**2)*(1.0 - F_upper)
    I_upper = (8.0 / delta**2)*(1.0 - F_lower)
    print("Approx. Lower bound on QFI =", I_lower)
    print("Approx. Upper bound on QFI =", I_upper)

example_demo()

Subfidelity E(rho, rho_delta) = 0.6999999999999995
Superfidelity R(rho, rho_delta) = 0.7000000000000002
Lower bound on fidelity = 0.8366600265340752
Upper bound on fidelity = 0.8366600265340757
Approx. Lower bound on QFI = 130.67197877273944
Approx. Upper bound on QFI = 130.6719787727398


In [30]:
n = 5 ## numero finale di qubit
N = 10 ## numero di qubit prima della traccia parziale

h_z_s = np.linspace(0, 5, num=20) # spazio dei valori di h_z

a_x= 3

delta = 0.5
derivative_delta = 1e-5
m = 7
DEBUG=True
initial_state = helper_functions.random_mixed_density_matrix(N, n)

# ket = qutip.rand_ket(2**n, distribution="haar").full().flatten()

y_s = np.array([pipeline_v200.simulation(
                        N=N,
                        n=n,
                        initial_state= initial_state,
                        DEBUG=True,
                        derivative_delta=derivative_delta,
                        a_x= a_x, 
                        h_z= h_z, ## variable
                        delta = delta,
                        m = m) for h_z in h_z_s])

# just for plotting clarity
sparse_indices = np.array([i for i in list(range(len(h_z_s))) if i%10==0])

Generating random mixed density matrix
purity of initial state = (0.06472190515913621+4.404571325722362e-20j) 
Dimension of the Hilbert space: 32
Initial state type: <class 'numpy.ndarray'> | initial_state: [[ 0.03673011-7.65176553e-20j  0.00750906-7.86573748e-03j
  -0.00591892+7.22096251e-03j ... -0.00337711-3.88459967e-03j
  -0.00774849-3.08871264e-03j  0.00988956-7.98889400e-03j]
 [ 0.00750906+7.86573748e-03j  0.03577378-1.81623967e-20j
  -0.002517  +2.01891967e-04j ... -0.00051985-5.61600218e-03j
   0.00462158-1.86042582e-03j -0.00124943+7.92116949e-03j]
 [-0.00591892-7.22096251e-03j -0.002517  -2.01891967e-04j
   0.02112163-4.71292834e-20j ... -0.00388465-3.61079798e-03j
   0.00121904-3.60329747e-04j -0.00200298-1.86560353e-03j]
 ...
 [-0.00337711+3.88459967e-03j -0.00051985+5.61600218e-03j
  -0.00388465+3.61079798e-03j ...  0.03076244+4.30606330e-20j
  -0.00305834-5.25915551e-03j -0.00497396+2.40581871e-03j]
 [-0.00774849+3.08871264e-03j  0.00462158+1.86042582e-03j
   0.00121904+